In [3]:
embeding_matrix = load_pickle_file("./embeding_matrix")
word2index = load_pickle_file("./word2index")
index_sequences = load_pickle_file("./index_sequences.pkl")
labels = load_pickle_file("./labels")
SEQUENCE_LENGTH = 618
VOCAB_SIZE = len(embeding_matrix)
EMB_DIMENSION = 300
VALIDATION_SPLIT = 0.1
PAD_SEQ_VALUE = 3001066


In [4]:
import numpy as np
same_lenght_seq = [seq[:SEQUENCE_LENGTH].reshape(1,SEQUENCE_LENGTH) if len(seq) >= SEQUENCE_LENGTH 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH) 
                   for seq in index_sequences]
same_lenght_seq = np.concatenate(same_lenght_seq,axis=0)


In [5]:
import random
test_size = int(len(labels) * 0.2)
true_labels = np.where(np.array(labels)==0)[0]
start_index = true_labels[0]
random_test_true_indexes = [int(x * len(true_labels)+start_index) for x in np.random.random(test_size//2)]
random_test_false_indexes = [int(x * (len(labels) - len(true_labels))) for x in np.random.random(test_size//2)]
x_test_indexes = random_test_true_indexes + random_test_false_indexes
random.shuffle(x_test_indexes)

x_train_indexes = [x for x in range(0,len(labels)) if x not in x_test_indexes]

random.shuffle(x_train_indexes)

labels = np.vstack(np.array([1,0]).reshape(1,2) if x == 0 else np.array([0,1]).reshape(1,2) for x in labels)

y_test = np.array(labels)[x_test_indexes]
y_train = np.array(labels)[x_train_indexes]

x_train = same_lenght_seq[x_train_indexes]
x_test = same_lenght_seq[x_test_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


Dataset split

In [4]:
# from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(same_lenght_seq, labels, test_size=0.2, random_state=42,shuffle = True)

In [6]:
data_size = len(x_train)
perm = np.random.permutation(data_size)
idx_train = perm[:int(data_size*(1-VALIDATION_SPLIT))]
idx_val = perm[int(data_size*(1-VALIDATION_SPLIT)):]

data_train = x_train[idx_train]
labels_train = y_train[idx_train]

data_val = x_train[idx_val]
labels_val = y_train[idx_val]

In [13]:
data_train.shape

(7380, 434)

<h1>Convolutional neural networks</h1>

In [7]:
import tensorflow as tf
from tensorflow import keras as tk
from tensorflow.keras import layers as tfkl
from sklearn.metrics import mean_squared_error,mean_absolute_error

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Conv1D, MaxPool1D, SpatialDropout1D, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [7]:
model_checkpoint = ModelCheckpoint("./Checkpoints", save_best_only=True, save_weights_only=True)


In [13]:
NUM_FILTERS = 35
KERNEL_SIZE = 20
rate_drop_lstm = 0.1
rate_drop_lstm = 0.1
LSTM_HIDDEN_DIM_SIZE = 10
model = tk.Sequential()
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH,
        trainable=False)
model.add(embedding_layer)
# conv_model.add(SpatialDropout1D())
model.add(Conv1D(NUM_FILTERS, KERNEL_SIZE, activation = 'relu',kernel_regularizer =tf.keras.regularizers.l2(l=0.01)))
model.add(Dropout(0.2))
# conv_model.add(BatchNormalization())
model.add(MaxPool1D(pool_size = 12))
model.add(LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'relu', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))

# conv_model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))


In [12]:
del model

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 618, 300)          900320100 
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 601, 25)           135025    
_________________________________________________________________
dropout_2 (Dropout)          (None, 601, 25)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 50, 25)            0         
_________________________________________________________________
unified_lstm_2 (UnifiedLSTM) (None, 8)                 1088      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 18        
Total params: 900,456,231
Trainable params: 136,131
Non-trainable params: 900,320,100
__________________________________

In [15]:
del model

In [11]:
model.reset_states()

In [9]:
opt = tk.optimizers.Adam(learning_rate=0.01)

In [14]:
model.compile(loss='binary_crossentropy',
        optimizer=opt,
        metrics=['acc'])
history2 = model.fit(data_train,labels_train,epochs = 25,batch_size = 256, validation_data = [data_val,labels_val],shuffle=True)

Train on 11732 samples, validate on 1304 samples
Epoch 1/25
11732/11732 [==============================] - 53s 4ms/sample - loss: 1.5436 - acc: 0.7070 - val_loss: 0.6141 - val_acc: 0.7408
Epoch 2/25
11732/11732 [==============================] - 54s 5ms/sample - loss: 0.5955 - acc: 0.7260 - val_loss: 0.5714 - val_acc: 0.7408
Epoch 3/25
11732/11732 [==============================] - 52s 4ms/sample - loss: 0.6022 - acc: 0.7243 - val_loss: 0.5947 - val_acc: 0.7408
Epoch 4/25
11732/11732 [==============================] - 53s 5ms/sample - loss: 0.5909 - acc: 0.7260 - val_loss: 0.5716 - val_acc: 0.7408
Epoch 5/25
11732/11732 [==============================] - 53s 5ms/sample - loss: 0.5867 - acc: 0.7260 - val_loss: 0.5730 - val_acc: 0.7408
Epoch 6/25
11732/11732 [==============================] - 52s 4ms/sample - loss: 0.5880 - acc: 0.7260 - val_loss: 0.5732 - val_acc: 0.7408
Epoch 7/25
11732/11732 [==============================] - 52s 4ms/sample - loss: 0.5872 - acc: 0.7260 - val_loss: 0.5

In [21]:
model.save("./Checkpoints/conv_model2.h5")

In [22]:
model.save_weights("./Checkpoints/conv_weights2")

In [31]:
len(np.where(np.argmax(y_test,axis=1) == 0)[0])

1583

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pred = model.predict(x_test)
pred = [np.argmax(x) for x in pred]

print(f"F1-score : {f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)}")
print(f"Accuracy-score : {accuracy_score([np.argmax(x) for x in y_test],pred)}")

print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")

F1-score : 0.9060422087308471
Accuracy-score : 0.8973468098547063
Precision : 0.8352878464818764
Recall  : 0.9898926089703095


In [29]:
from sklearn.metrics import accuracy_score

pred = conv_model.predict(x_test)
pred = [np.argmax(x) for x in pred]



In [30]:
from sklearn.metrics import f1_score
f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)

0.9362930077691454

In [31]:
accuracy_score([np.argmax(x) for x in y_test],pred)

0.9093779602147143

In [32]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

0.8797661587489404

In [34]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 0)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 0)}")

Precision : 0.8851894374282434
Recall  : 0.8048016701461378


In [2]:
import pickle
def save_pickle_file(file,file_path,protocol=None):
    with open(file_path,"wb") as f:
        if protocol:
            pickle.dump(file,f,protocol = protocol)
        else:
            pickle.dump(file,f)

def load_pickle_file(file_path, protocol=None):
     with open(file_path,"rb") as f:
        if protocol:
            return pickle.load(f)
        else:
            return pickle.load(f)
